In [1]:
String jsonText = new File('data/api-export/genes/genes-673.json').text
def slurper = new groovy.json.JsonSlurper()
json =slurper.parseText(jsonText)
OutputCell.HIDDEN

In [2]:
//  impact color map for lollies

def VARIANT_IMPACT_COLORS = ["deletion": '#ED8B00', "duplication": '#333F48', "frameshift": '#A2AAAD',
                                 "fusion"  : '#0085CA', "indel": '#9467bd', "insertion": '#43B02A', "missense": '#002D72',
                                 "none"    : '#7f7f7f', "nonsense": '#bcbd22', "splice site": '#17becf', "unknown": '#D0D3D4']

In [3]:
// get maps of fields
mapByImpact = json.variants.groupBy{it.impact}// factor
mapByName = json.variants.groupBy{it.fullName}
OutputCell.HIDDEN

In [9]:
// build up genetic mutation data object by extracting relevant fields
def impacts = mapByImpact.keySet() as List
def impactCounts = mapByImpact.values().collect{it.size()}
def names = mapByName.keySet() as List

//println("names :"+names)

// regular expression matching to extract position
def builder = new groovy.json.JsonBuilder(json)
builder.content.variants.each{
    def fullName = it.fullName
    def group = fullName =~ /.* \w([0-9]+)[^0-9]+/  
    if(group){
        def position = group[0][1]
        it.position = position
    }
    else{
        it.position = null
    }
    //println("position :"+it.position+"\n")
} 

// test print (quality check)
def map = builder.content.variants[0]
println("example of variant zero (0) object data ->\n\n"+"fullname: ${map.fullName} ==> position: ${map.position}")
println("x : "+map.position+"\ny :"+map.id+"\nfactor :"+map.impact )
println("\n\nFull object first variable format: \n"+builder.content.variants[0])

// updated JSON file
new File('data/api-export/genes/genes-673-with-position.json').text = builder.toPrettyString()

// build Genetic mutation data object
def mutation_data = builder.content.variants

// new JSON file of just variants
//def slurper = new groovy.json.JsonSlurper()
//json =slurper.parseText(mutation_data)
//new File('data/api-export/genes/genes-673-variants.json').text = mutation_data


beakerx.mutation_data = mutation_data

println ("\n\nmutation data object quality check :\n\n"+"size : "+mutation_data.size()+"\n\n"+mutation_data[0])

OutputCell.HIDDEN

example of variant zero (0) object data ->

fullname: BRAF S602F ==> position: 602
x : 602
y :27728
factor :missense


Full object first variable format: 
[id:27728, fullName:BRAF S602F, impact:missense, proteinEffect:no effect - predicted, geneVariantDescriptions:[[description:BRAF S602F lies within the protein kinase domain of the Braf protein (UniProt.org). S602F has not been biochemically characterized, but results in similar cell proliferation and viability levels to wild-type Braf in culture (PMID: 29533785), and therefore is predicted to have no effect on Braf protein function., references:[[id:11238, pubMedId:29533785, title:Systematic Functional Annotation of Somatic Mutations in Cancer., url:http://www.ncbi.nlm.nih.gov/pubmed/29533785], [id:295, pubMedId:null, title:null, url:www.uniprot.org]]]], position:602]


mutation data object quality check :

size : 263

[id:27728, fullName:BRAF S602F, impact:missense, proteinEffect:no effect - predicted, geneVariantDescriptions:[[desc

In [5]:
%%javascript

var mutation_data_default_settings = {
    x: "position",         // mutation position
    y: "fullname",      // impact count
    factor: "impact", // classify mutations by certain factor (optional)
};

beakerx.mutation_data_default_settings = mutation_data_default_settings

In [6]:
%%javascript


var d3 = require(['d3'], function (d3) {
    console.log("hello d3" + d3);
});
// generate g3-lollipop chart

// create in "g3lollipop" div
var g3 = require(['g3'], function (g3) {
    console.log("hello g3" + g3);
    var lollipop = g3.Lollipop("g3lollipop");

    // add mutation data
    lollipop.data.snvData = beakerx.mutation_data;
    // mutation data format settings
    lollipop.format.snvData = mutation_data_default_settings;

    // Pfam domain data
    lollipop.data.domainData = pfam_data;
    // Pfam data format settings
    lollipop.format.domainData = pfam_data_default_settings;

    // set up more chart options ...
    var plot_options = {
        "chartMargin": {
            "left": 40, "right": 40, "top": 30, "bottom": 25
        },
        "titleFont": "normal 20px Sans",
        "titleColor": "steelblue",
        "titleAlignment": "middle",
        "titleDy": "0.3em"
    };

    // add chart options
    lollipop.setOptions(plot_options);

    lollipop.options.chartMargin = {
    "left": 40, "right": 40, "top": 30, "bottom": 25};
    lollipop.options.titleFont = "normal 20px Sans";
    lollipop.options.titleColor = "steelblue";
    lollipop.options.titleAlignment = "middle";
    lollipop.options.titleDy = "0.3em";
    
    // draw lollipop
    lollipop.draw();

});


In [7]:
%%html

<!DOCTYPE html>
<html>

<head>
    <meta charset="utf-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <link rel="stylesheet" href="https://s3-us-west-2.amazonaws.com/cdsjsd3/g3-viz/0.5.0/g3-lollipop.min.css">
    <link rel="stylesheet" href="https://s3-us-west-2.amazonaws.com/cdsjsd3/css/gist.css">
</head>

<body>
    <div>
        <div>
            <span class="btn-group">
                <button id="save-as-png">save as PNG</button>
                <button id="save-as-svg">save as SVG</button>
            </span>
        </div>

        <div id="lollipop"></div>
    </div>

    <script src="https://d3js.org/d3.v4.min.js"></script>
    <script src="https://s3-us-west-2.amazonaws.com/cdsjsd3/g3-viz/0.5.0/g3-lollipop.min.js"></script>
    <script>
        // mutation data in TSV (tabular-separated value) format
        var snvFile = "TP53-msk_impact_2017-parsed.tsv";

        // pfam domain data
        var domainFile = "TP53_pfam.json";

        // mutation data format [to be modified]
        var snvDataFormat = {
            x: "AA_Position",
            y: "Protein_Change",
            factor: "Mutation_Type",
        };

        // pfam domain JSON information [to be modified]
        var domainDataFormat = {
            symbol: "hgnc_symbol",
            name: "protein_name",
            length: "length",
            domainType: "pfam",
            details: {
                start: "pfam_start",
                end: "pfam_end",
                ac: "pfam_ac",
                name: "pfam_id",
            },
        };

        // read data in
        var q = d3.queue();
        q.defer(d3.tsv, snvFile);
        q.defer(d3.json, domainFile);

        q.await(function (error, snvData, domainData) {
            // format data
            snvData.forEach(function (d) {
                d[snvDataFormat.x] = +d[snvDataFormat.x];
            });

            // new lollipop chart
            var lollipop = g3.Lollipop("lollipop");

            // add data
            lollipop.data.snvData = snvData;
            lollipop.data.domainData = domainData;
            // add data options
            lollipop.format.snvData = snvDataFormat;
            lollipop.format.domainData = domainDataFormat;

            // draw with default settings
            lollipop.draw();

            // get chart div id
            var chartID = lollipop.options.chartID;

            // set default filename
            var output_chart_filename = "lollipop";

            // button actions: download chart
            document.getElementById("save-as-png").onclick = function (e) {
                g3.output().toPNG(output_chart_filename, chartID);
            };

            document.getElementById("save-as-svg").onclick = function (e) {
                g3.output().toSVG(output_chart_filename, chartID);
            };

        });
    </script>
</body>

</html>



<!DOCTYPE html>
 

 
 
 
 
 
 

 
 
 
 
 save as PNG 
 save as SVG